# Deterministic Models

In [13]:
library(psych)

# Exercise 3

To develop an algorithm capable of beating general-purpose frameworks in this concrete problem, we have leverage on the following ideas:


- __Coordinate-wise vs Gradient Optimization:__ Gradient is faster if it's not taken into account the time to calculate the gradient. In this type of problem, the cost of this step grows quadratically with the size of the matrix $(n)$.


- __Speeding up the calculation of the function:__ The function to maximize $ f(x) = \log(\det(K)) - tr(SK) $ is also computationally  expensive. The determinant of a matrix grows as $n!$. Taking advantage of the former decision  (coordenate-wise optimization), condition  on $x_{ii}, \det(K)$ it's a linear function and condition on $x_{ij}, \det(K)$ is a quadratic function. Thus once we've chosen the coordinate we will optimize on, we precalculate the function $\det(K)$ as a function of $x_{ij}$ or $x_{ii}$.


- __Turning the problem into an unconstrained problem__: The original optimization problem is constrained to $x_{ij} \leq 0$ and $ x_{ij} \geq 0$ insted of optimizing  $f(x_{ij}$ we will optimize $f(-x_{ij}^2$ and instead of optimizing $f(x_{ii})$ we will optimize $f(x_{ii}^2)$


- __Choosing the initial point:__ The original problem is concave. The unconstrained version $f(-x_{ij}^2), f(x_{ii}^2)$ has $\frac{(n-1)n}{2}$ minimums. In particular diagonal matrices $D$ lay in the boundary of two convergence regions. Thus these are the type of point not to start with. We have chosen $Id - \epsilon $


### So let's get started

### Poli_det

This function generates the polynomial expression of $\det(K)$ condition to $x_{ij}$


In [14]:
poli_det <- function(K, i, j){
  if (i==j){
    # Calculate
    K[i, j] <- 0
    V0 <- det(K)
    K[i, j] <- 1
    V1 <- det(K)
    
    # Solve
    c <- V0
    a <- V1 - c
    
    f <- function(x){
      a*x + c
    }
  }else{
    # Calculate
    K[i, j] <- 0
    K[j, i] <- 0
    V0 <- det(K)
    K[i, j] <- 1
    K[j, i] <- 1
    V1 <- det(K)
    K[i, j] <- -1
    K[j, i] <- -1
    V_1 <- det(K)
    
    # Solve
    c <- V0
    a <- (V1 + V_1 - 2*c)/2
    b <- V1 - c - a
    
    f <- function(x){
      a*x^2+b*x++c
    }
  }
  f
}

### Poli_trace
Calculates the polinomyal expression of $tr(SK)$ condition on $x{ij}$

In [15]:
poli_trace <- function(S, K, i, j){
  if(i==j){
    # Calculate
    K[i, j]<- 0
    V0 <- tr(S%*%K)
    K[i, j]<- 1
    V1 <- tr(S%*%K)
    
    # Solve
    b <- V0
    a <- V1 - b
    
    f <- function(x){
      a*x+b
    }
  }else{
    # Calculate
    K[i, j] <- 0
    K[j, i] <- 0
    V0 <- tr(S%*%K)
    K[i, j] <- 1
    K[j, i] <- 1
    V1 <- tr(S%*%K)

    # Solve
    b <- V0
    a <- V1 - b
    
    f <- function(x){
      a*x+b
    }
  }
}

### Poli_fun

Generates the quasi-polynomial function equivalent to our objective function condition on $x_{ij}$

In [16]:
poli_fun <- function(S, K, i, j){
  pd <- poli_det(K, i, j)
  pt <- poli_trace(S, K, i, j)
  if (i==j){
    function(x){
      log(pd(x^2))-pt(x^2)
    }
  }else{
    function(x){
      log(pd(-x^2))-pt(-x^2)
    }
  }
}

### Poli_par

The following pair of functions calculate the partial derivatives of $f(x)$. This functions will be used to choose which is the best coordinate to performe the optimization

In [17]:
poli_par <- function(f, x0){
  eps = 0.0001
  (f(x0+eps)-f(x0-eps))/(2*eps)
}

poli_par_ij <- function(S, K, i, j){
  f <- poli_fun(S, K, i, j)
  if (i==j){
    par <- poli_par(f, (K[i, j])^.5)
  }else{
    par <- poli_par(f, (-(K[i, j]))^.5)
  }
  par
}

### Find ij

This function chooses the best coordinate to perform the optimization

In [18]:
find_ij <- function(S, K){
  n <- dim(K)[1]
  
  the_i <- 0
  the_j <- 0
  the_v <- -Inf
  
  for (i in 1:n){
    for (j in i:n){
      if (i == j){
        new_v <- abs(poli_par(poli_fun(S, K, i, j), (K[i, j])^.5))
      }else{
        new_v <- abs(poli_par(poli_fun(S, K, i, j), (-K[i, j])^.5))
      }
      
      if (new_v > the_v){
        the_v <- new_v
        the_i <- i
        the_j <- j
      }
    }
  }
  c(the_i, the_j)
}


### Setting all together

Finally we are able to define a procedure that beats a general-purpose optimizer even in low dimensions

In [19]:
# Optimization on coordinate x_{ij}
optimize_f_ij <- function(S, K, i, j){
  f <- poli_fun(S, K, i, j)
  
  if (i == j){
    x0 <- (K[i, j])^.5
  }else{
    x0 <- (-K[i, j])^.5
  }
  
  
  result <- optim(x0, f, control=list(fnscale=-1, reltol=1e-8))
  
  if(i == j){
    K[i,j] <- (result$par)^2
  }else{
    K[i, j] <- -(result$par)^2
    K[j, i] <- -(result$par)^2
  }
  K
}

# API 
exercise_3 <- function(S, pre = 1e-1, max_iter = 100){
  
  n <- dim(S)[1]
  
  eps <- 0.1
  
  # We set the right epsilon so det(K) > 0 
  while (TRUE){
    K <- diag(n)
    K <- K - eps*matrix(1, nrow = n, ncol = n) 
    
    if (det(K)>0){
      break
      }else{
        eps<-eps/2
      } 
  }

  
  last_value <- Inf
  
  # We make sure that at least each component is maximized once
  for (i in 1:n){
    for (j in i:n){
      aux <- optimize_f_ij(S, K, i, j)
      K <- aux
    }
  }
  
  # Here we could be fancyer
  for (mi in 1:max_iter){
    ij = find_ij(S, K)
    
    i <- ij[1]
    j <- ij[2]
    
    aux <- optimize_f_ij(S, K, i, j)
    K <- aux
  }
  round(K, 2)
}

### Testing

In [20]:


#set.seed(1)

n <- 9     ## Dimension of matrix
S <- matrix(rnorm(n*n), ncol=n)
S <- S%*%t(S)

n <- dim(S)[1]
K <- diag(n)

print('exercise_2')
print('this is going to be slow')
#Kr_2 <- exercise_2(S, pre=1e-2, max_iter = 1000)
#print(Kr_2)
print('exercise_3')
Kr_3 <- exercise_3(S, pre=1e-2, max_iter = 100)
print(Kr_3)
print('Done!!! ;-)')

[1] "exercise_2"
[1] "this is going to be slow"
[1] "exercise_3"


Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in log(pd(x^2)):
"NaNs produced"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"on

use "Brent" or optimize() directly"Warning message in log(pd(x^2)):
"NaNs produced"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in log(pd(-x^2)):
"NaNs produced"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in log(pd(x^2)):
"NaNs produced"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in log(pd(x^2)):
"NaNs produced"Warning message in log(pd(x^2)):
"NaNs produced"Warning message in optim(x0,

use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional

use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional optimization by Nelder-Mead is unreliable:
use "Brent" or optimize() directly"Warning message in optim(x0, f, control = list(fnscale = -1, reltol = 1e-08)):
"one-dimensional

       [,1]  [,2]  [,3]  [,4]  [,5]  [,6]  [,7]  [,8]  [,9]
 [1,]  0.31  0.00  0.00 -0.21  0.00  0.00  0.00 -0.02  0.00
 [2,]  0.00  0.19  0.00  0.00 -0.05  0.00  0.00  0.00  0.00
 [3,]  0.00  0.00  0.09  0.00  0.00 -0.05 -0.01  0.00  0.00
 [4,] -0.21  0.00  0.00  1.00  0.00  0.00  0.00 -0.27  0.00
 [5,]  0.00 -0.05  0.00  0.00  0.15  0.00  0.00 -0.03 -0.05
 [6,]  0.00  0.00 -0.05  0.00  0.00  0.35  0.00  0.00  0.00
 [7,]  0.00  0.00 -0.01  0.00  0.00  0.00  0.06  0.00  0.00
 [8,] -0.02  0.00  0.00 -0.27 -0.03  0.00  0.00  0.21 -0.01
 [9,]  0.00  0.00  0.00  0.00 -0.05  0.00  0.00 -0.01  0.13
[1] "Done!!! ;-)"
